# Zadania na 3.0 Julia


In [1]:
using DataStructures
using Distributions


In [2]:
struct KnapsackProblem
    capacities::Vector{Int}  
    weights::Vector{Int}     
    profits::Vector{Int}     
end

function objective(p::KnapsackProblem, x)
    total_profit = 0
    for i in 1:length(x)
        if x[i] > 0  
            total_profit += p.profits[i]
        end
    end
    return -total_profit
end


objective (generic function with 1 method)

In [3]:
function possible_moves(p::KnapsackProblem, x::Vector{Int})
    move_list = Tuple{Symbol, Int, Int}[]  
    
    for i in 1:length(x)
        if x[i] == 0  
            for knapsack_index in 1:3
                if sum(p.weights[x .== knapsack_index]) + p.weights[i] <= p.capacities[knapsack_index]
                    push!(move_list, (:add, i, knapsack_index))  
                end
            end
        elseif x[i] > 0  
            push!(move_list, (:remove, i, x[i]))  
        end
    end
    return move_list
end


function apply!(x, move::Tuple{Symbol, Int, Int})
    item_index = move[2]  
    knapsack_index = move[3]  
    
    if move[1] === :add
        x[item_index] = knapsack_index  
    elseif move[1] === :remove
        x[item_index] = 0 
    end
    return x
end


function invert_move(::KnapsackProblem, move::Tuple{Symbol, Int, Int})
    item_index = move[2]
    knapsack_index = move[3]
    
    if move[1] === :add
        return (:remove, item_index, knapsack_index)  
    else
        return (:add, item_index, knapsack_index) 
    end
end


mutable struct TabuState{TMove,P,TF}
    tabu_buffer::CircularBuffer{TMove}
    best_seen::P
    best_seen_obj::TF
    current::P
    considered::P
    iter::Int
end

function TabuState(p, x0; buffer_length::Int=10)
    moves = possible_moves(p, x0)
    obj = objective(p, x0)
    return TabuState{eltype(moves),typeof(x0),typeof(obj)}(
        CircularBuffer{eltype(moves)}(buffer_length), x0, obj, copy(x0), copy(x0), 1
    )
end

TabuState

In [4]:
function solve_tabu(p, s::TabuState; iteration_limit::Int=100)
    while s.iter < iteration_limit
        moves = possible_moves(p, s.current)  
        best_move = 0
        best_move_obj = Inf
        for (i_move, move) in enumerate(moves)
            if in(move, s.tabu_buffer)  
                continue
            end
            copyto!(s.considered, s.current)
            apply!(s.considered, move)  
            considered_value = objective(p, s.considered)
            if considered_value < best_move_obj  
                best_move = i_move
                best_move_obj = considered_value
            end
        end
        if best_move == 0  
            break
        end
        apply!(s.current, moves[best_move])  
        push!(s.tabu_buffer, invert_move(p, moves[best_move]))  
        if best_move_obj < s.best_seen_obj  
            copyto!(s.best_seen, s.current)
            s.best_seen_obj = best_move_obj
        end
        s.iter += 1
    end
    return s.best_seen
end


solve_tabu (generic function with 1 method)

In [5]:
function generate_problem_random()
    n_items = 10
    profits = rand(DiscreteUniform(10, 1000), n_items)
    weights = rand(DiscreteUniform(10, 100), n_items)
    capacities = rand(DiscreteUniform(180, 220), 3)
    return KnapsackProblem(capacities, weights, profits)
end


function generate_problem_1()
    profits = [26, 301, 644, 595, 436, 98, 943, 817, 86, 97]
    weights = [71, 25, 48, 11, 16, 83, 41, 98, 30, 24]
    capacities = [210, 200, 195]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_2()
    profits = [943, 359, 544, 579, 16, 85, 756, 235, 165, 131]
    weights = [35, 47, 53, 56, 58, 35, 22, 14, 64, 53]
    capacities = [187, 214, 214]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_3()
    profits = [750, 252, 141, 407, 537, 431, 262, 188, 614, 749, 470]
    weights = [33, 69, 35, 59, 15, 12, 21, 42, 28, 46, 10]
    capacities = [202, 206, 187]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_4()
    profits = [325, 436, 534, 999, 414, 602, 979, 884, 587, 142, 60, 888, 478]
    weights = [54, 32, 27, 12, 89, 87, 77, 75, 65, 32, 85, 49, 16]
    capacities = [188, 185, 209]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_5()
    profits = [100, 736, 410, 729, 424, 800, 877, 307, 310, 93, 870, 899, 281, 220]
    weights = [13, 92, 31, 26, 71, 55, 87, 77, 74, 43, 46, 20, 22, 59]
    capacities = [220, 208, 191]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_6()
    profits = [970, 371, 154, 742, 857, 825, 858, 323, 748, 747, 240, 624, 727, 606, 644]
    weights = [68, 12, 26, 24, 14, 30, 85, 34, 95, 13, 34, 82, 23, 11, 60]
    capacities = [216, 220, 183]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_7()
    profits = [877, 58, 749, 106, 612, 133, 805, 155, 62, 416, 267, 150, 313, 383, 378, 55]
    weights = [68, 87, 59, 61, 21, 99, 39, 45, 66, 68, 77, 87, 61, 11, 50, 23]
    capacities = [211, 209, 211]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_8()
    profits = [549, 847, 968, 209, 917, 486, 975, 753, 14, 501, 951, 357, 953, 550, 190, 594, 267]
    weights = [18, 84, 77, 94, 34, 91, 56, 92, 47, 26, 55, 89, 97, 50, 19, 28, 100]
    capacities = [201, 207, 207]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_9()
    profits = [285, 550, 964, 952, 899, 862, 898, 975, 58, 661, 750, 350, 660, 89, 387, 378, 620, 237]
    weights = [10, 26, 55, 75, 39, 85, 96, 62, 57, 46, 47, 47, 84, 52, 24, 37, 51, 77]
    capacities = [187, 181, 188]
    return KnapsackProblem(capacities, weights, profits)
end

function generate_problem_10()
    profits = [971, 338, 656, 639, 578, 507, 496, 135, 981, 841, 62, 934, 829, 192, 128, 168, 47, 81]
    weights = [23, 28, 42, 19, 49, 88, 88, 27, 35, 39, 92, 12, 39, 26, 43, 93, 27, 96]
    capacities = [192, 209, 219]
    return KnapsackProblem(capacities, weights, profits)
end

generate_problem_10 (generic function with 1 method)

In [6]:
using BenchmarkTools

function test(kp)
    for tabu_lenght in [1, 2, 5]

        println("\nTabu length: ", tabu_lenght)

        x0 = fill(0, length(kp.weights))  
        st = TabuState(kp, x0; buffer_length=tabu_lenght)
        @time begin
            sol = solve_tabu(kp, st; iteration_limit=1000000)
        end

        plecak1 = []
        plecak2 = []
        plecak3 = []
        profit = 0
        weight1 = 0
        weight2 = 0
        weight3 = 0

        for i in 1:length(sol)
            if sol[i] == 1
                push!(plecak1, i)
                profit += kp.profits[i]
                weight1 += kp.weights[i]  
            elseif sol[i] == 2
                push!(plecak2, i)
                profit += kp.profits[i]
                weight2 += kp.weights[i]  
            elseif sol[i] == 3
                push!(plecak3, i)
                profit += kp.profits[i]
                weight3 += kp.weights[i]  
            end
        end
        
        println("Plecak 1: ", plecak1, " - Profit: ", sum(kp.profits[plecak1]), " - Waga: ", weight1, "/", kp.capacities[1])
        println("Plecak 2: ", plecak2, " - Profit: ", sum(kp.profits[plecak2]), " - Waga: ", weight2, "/", kp.capacities[2])
        println("Plecak 3: ", plecak3, " - Profit: ", sum(kp.profits[plecak3]), " - Waga: ", weight3, "/", kp.capacities[3])
        println("Łączny profit: ", profit)
    end
end


test (generic function with 1 method)

In [7]:
println("Zestaw 8")
kp8 = generate_problem_8()
test(kp8)

println("\nZestaw 9")
kp9 = generate_problem_9()
test(kp9)

println("\nZestaw 10")
kp10 = generate_problem_10()
test(kp10)

Zestaw 8

Tabu length: 1
  2.225879 seconds (108.00 M allocations: 4.724 GiB, 26.27% gc time)
Plecak 1: Any[3, 7, 11] - Profit: 2894 - Waga: 188/201
Plecak 2: Any[1, 5, 10, 13, 16] - Profit: 3514 - Waga: 203/207
Plecak 3: Any[2, 8, 15] - Profit: 1790 - Waga: 195/207
Łączny profit: 8198

Tabu length: 2
  2.396286 seconds (110.50 M allocations: 4.788 GiB, 25.90% gc time)
Plecak 1: Any[3, 7, 11] - Profit: 2894 - Waga: 188/201
Plecak 2: Any[1, 5, 10, 13, 16] - Profit: 3514 - Waga: 203/207
Plecak 3: Any[2, 8, 15] - Profit: 1790 - Waga: 195/207
Łączny profit: 8198

Tabu length: 5
  2.166068 seconds (97.50 M allocations: 4.412 GiB, 25.54% gc time)
Plecak 1: Any[3, 7, 11] - Profit: 2894 - Waga: 188/201
Plecak 2: Any[1, 5, 10, 13, 16] - Profit: 3514 - Waga: 203/207
Plecak 3: Any[2, 8, 15] - Profit: 1790 - Waga: 195/207
Łączny profit: 8198

Zestaw 9

Tabu length: 1
  2.579294 seconds (130.50 M allocations: 5.497 GiB, 26.23% gc time)
Plecak 1: Any[2, 3, 5, 8] - Profit: 3388 - Waga: 182/187
Plecak

# Zadania na 4.0

In [8]:
function possible_moves(p::KnapsackProblem, x::Vector{Int})
    move_list = Tuple{Symbol, Int, Int}[]  
    
    for i in 1:length(x)
        if x[i] == 0  
            for knapsack_index in 1:3
                if sum(p.weights[x .== knapsack_index]) + p.weights[i] <= p.capacities[knapsack_index]
                    push!(move_list, (:add, i, knapsack_index))  
                end
            end
        elseif x[i] > 0  
            push!(move_list, (:remove, i, x[i]))  
            
            for target_knapsack in 1:3
                if target_knapsack != x[i] 
                    if sum(p.weights[x .== target_knapsack]) + p.weights[i] <= p.capacities[target_knapsack]
                        push!(move_list, (:move, i, target_knapsack))  
                    end
                end
            end
        end
    end
    return move_list
end

function apply!(x, move::Tuple{Symbol, Int, Int})
    item_index = move[2]  
    knapsack_index = move[3]  
    
    if move[1] === :add
        x[item_index] = knapsack_index  
    elseif move[1] === :remove
        x[item_index] = 0  
    elseif move[1] === :move
        x[item_index] = knapsack_index  
    end
    return x
end

function invert_move(::KnapsackProblem, move::Tuple{Symbol, Int, Int})
    item_index = move[2]
    knapsack_index = move[3]
    
    if move[1] === :add
        return (:remove, item_index, knapsack_index)  
    elseif move[1] === :remove
        return (:add, item_index, knapsack_index)  
    elseif move[1] === :move
        return (:move, item_index, knapsack_index)  
    end
end


invert_move (generic function with 1 method)

In [9]:
println("Zestaw 1")
kp1 = generate_problem_1()
test(kp1)

println("Zestaw 2")
kp2 = generate_problem_2()
test(kp2)

println("\nZestaw 3")
kp3 = generate_problem_3()
test(kp3)

println("\nZestaw 4")
kp4 = generate_problem_4()
test(kp4)

println("Zestaw 5")
kp5 = generate_problem_5()
test(kp5)

println("\nZestaw 6")
kp6 = generate_problem_6()
test(kp6)

println("\nZestaw 7")
kp7 = generate_problem_7()
test(kp7)

println("Zestaw 8")
kp8 = generate_problem_8()
test(kp8)

println("\nZestaw 9")
kp9 = generate_problem_9()
test(kp9)

println("\nZestaw 10")
kp10 = generate_problem_10()
test(kp10)



Zestaw 1

Tabu length: 1
  2.697043 seconds (103.00 M allocations: 4.560 GiB, 25.94% gc time)
Plecak 1: Any[3, 4, 7, 8] - Profit: 2999 - Waga: 198/210
Plecak 2: Any[2, 5, 6, 9, 10] - Profit: 1018 - Waga: 178/200
Plecak 3: Any[1] - Profit: 26 - Waga: 71/195
Łączny profit: 4043

Tabu length: 2
  2.553491 seconds (103.00 M allocations: 4.570 GiB, 24.80% gc time)
Plecak 1: Any[3, 4, 7, 8] - Profit: 2999 - Waga: 198/210
Plecak 2: Any[2, 5, 6, 9, 10] - Profit: 1018 - Waga: 178/200
Plecak 3: Any[1] - Profit: 26 - Waga: 71/195
Łączny profit: 4043

Tabu length: 5
  2.502570 seconds (103.00 M allocations: 4.562 GiB, 23.22% gc time)
Plecak 1: Any[3, 4, 7, 8] - Profit: 2999 - Waga: 198/210
Plecak 2: Any[2, 5, 6, 9, 10] - Profit: 1018 - Waga: 178/200
Plecak 3: Any[1] - Profit: 26 - Waga: 71/195
Łączny profit: 4043
Zestaw 2

Tabu length: 1
  2.310116 seconds (103.00 M allocations: 4.560 GiB, 24.72% gc time)
Plecak 1: Any[1, 3, 4, 7, 8] - Profit: 3057 - Waga: 180/187
Plecak 2: Any[2, 6, 9, 10] - Prof

InterruptException: InterruptException: